In [3]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
url = "https://www.amazon.co.uk/Apple-iPhone-15-128-GB/dp/B0CHX7Z69Z/ref=sr_1_1_sspa?crid=1ZNTHJFE8S6MR&keywords=iphone&qid=1698077516&sprefix=iphon%2Caps%2C659&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"

# Send a GET request to the URL
response = requests.get(url, headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the element with the specified ID and class
product_title_element = soup.find('span', {'id': 'productTitle', 'class': 'a-size-large product-title-word-break'})

# Get the text from the element
product_title = product_title_element.get_text(strip=True)

print(product_title)


AttributeError: 'NoneType' object has no attribute 'get_text'

In [4]:
response

<Response [200]>